In [25]:
!pip install qiskit-nature -q
!pip install qiskit-nature-pyscf -q

In [26]:
!pip install qgear-lightning -q
!pip install pylatexenc

In [27]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator
from qgear_lightning.core import qiskit_to_gateList, counts_cudaq_to_qiskit, circ_kernel
from qgear_lightning.io import write4_data_hdf5, read4_data_hdf5
import cudaq
# Use PySCF, a classical computational chemistry software
# package, to compute the one-body and two-body integrals in
# electronic-orbital basis, necessary to form the Fermionic operator
driver = PySCFDriver(
    atom='H .0 .0 .0; H .0 .0 0.735',
    unit=DistanceUnit.ANGSTROM,
    basis='sto3g',
)
problem = driver.run()

# setup the qubit mapper
from qiskit_nature.second_q.mappers import ParityMapper

mapper = ParityMapper(num_particles=problem.num_particles)

# setup the classical optimizer for the VQE
from qiskit_algorithms.optimizers import L_BFGS_B

optimizer = L_BFGS_B()

from qiskit.primitives import StatevectorEstimator

estimator = StatevectorEstimator()

# setup the ansatz for VQE
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

ansatz = UCCSD(
    problem.num_spatial_orbitals,
    problem.num_particles,
    mapper,
    initial_state=HartreeFock(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper,
    )
)

ansatz.decompose(reps=5).draw('mpl')
print(problem.hamiltonian)


In [28]:

def expect_cudaq(gateD, hamiltonian, backend="qpp-cpu"):
    """
    Run CUDA-Q simulation for all circuits in gateD and compute expectation values.

    Parameters
    ----------
    gateD : dict
        Dictionary containing circuit definitions ('circ_type', 'gate_type', 'gate_param').
    hamiltonian : cudaq.SpinOperator
        Hamiltonian for expectation value calculation.
    verb : int
        Verbosity level. >1 prints all circuits, 1 prints only first small one.
    backend : str
        CUDA-Q backend target (e.g., 'qpp-cpu', 'nvidia').

    Returns
    -------
    exp_vals : list of float
        Expectation values for each circuit.
    """
    cudaq.set_target(backend)
    nc = len(gateD['circ_type'])
    exp_vals = [0.0] * nc

    for i in range(nc):
        num_qubit, num_gate = map(int, gateD['circ_type'][i])
        gate_type = list(map(int, gateD['gate_type'][i].flatten()))
        gate_param = list(map(float, gateD['gate_param'][i].flatten()))
        assert num_gate <= len(gate_param)

        # Compute expectation value
        result = cudaq.observe(circ_kernel, hamiltonian, num_qubit, num_gate, gate_type, gate_param)
        exp_vals[i] = result.expectation()

    return exp_vals

In [29]:
def cost_func_vqe_cudaq(params, ansatz_template, hamiltonian, estimator):
    # Bind new parameters into the ansatz
    qc_bound = ansatz_template.assign_parameters(params)

    # Convert to CUDA-Q gate list
    gateD, md = qiskit_to_gateList([qc_bound])

    # Evaluate expectation value
    cost = expect_cudaq(gateD, hamiltonian)
    return cost

In [30]:
from cudaq import spin

def qiskit_to_cudaq(qiskit_op):
    cudaq_op = 0.0
    for pauli, coeff in zip(qiskit_op.paulis, qiskit_op.coeffs):
        term = 1.0
        for idx, p in enumerate(str(pauli)):
            if p == 'X':
                term *= spin.x(idx)
            elif p == 'Y':
                term *= spin.y(idx)
            elif p == 'Z':
                term *= spin.z(idx)
        cudaq_op += coeff.real * term
    return cudaq_op
second_q_op = problem.hamiltonian.second_q_op()
hamiltonian = mapper.map(second_q_op)
observable_1 = qiskit_to_cudaq(hamiltonian)

In [31]:
qc = ansatz.decompose(reps=5)

In [32]:
from scipy.optimize import minimize
import time
import numpy as np


x0 = np.random.rand(qc.num_parameters)
start_time = time.time()

result = minimize(
    cost_func_vqe_cudaq,
    x0,
    args=(qc, observable_1, estimator),
    method="COBYLA",
    options={"maxiter": 1000, "disp": True},
)

print("Optimal parameters:", result.x)
print("Minimum energy:", result.fun)
print("Time taken:", time.time() - start_time)

qiskit_to_gateList: nGate 25
qiskit_to_gateList: nGate 25
qiskit_to_gateList: nGate 25
qiskit_to_gateList: nGate 25
qiskit_to_gateList: nGate 25
qiskit_to_gateList: nGate 25
qiskit_to_gateList: nGate 25
qiskit_to_gateList: nGate 25
qiskit_to_gateList: nGate 25
qiskit_to_gateList: nGate 25
qiskit_to_gateList: nGate 25
qiskit_to_gateList: nGate 25
qiskit_to_gateList: nGate 25
qiskit_to_gateList: nGate 25
qiskit_to_gateList: nGate 25
qiskit_to_gateList: nGate 25
qiskit_to_gateList: nGate 25
qiskit_to_gateList: nGate 25
qiskit_to_gateList: nGate 25
qiskit_to_gateList: nGate 25
qiskit_to_gateList: nGate 25
qiskit_to_gateList: nGate 25
qiskit_to_gateList: nGate 25
qiskit_to_gateList: nGate 25
qiskit_to_gateList: nGate 25
qiskit_to_gateList: nGate 25
qiskit_to_gateList: nGate 25
qiskit_to_gateList: nGate 25
qiskit_to_gateList: nGate 25
qiskit_to_gateList: nGate 25
qiskit_to_gateList: nGate 25
qiskit_to_gateList: nGate 25
qiskit_to_gateList: nGate 25
qiskit_to_gateList: nGate 25
qiskit_to_gate